# Building an AI-powered multimodal RAG system with Docling
*Using IBM Granite vision, text-based embeddings and generative AI models*


---


## Understanding Multimodal RAG with Visual Grounding


---


### Technologies We'll Use

Building on our previous labs, we'll add:

1. **[Docling](https://docling-project.github.io/docling/):** An open-source toolkit used to parse and convert documents.
2. **[LangChain](https://langchain.com)**: For orchestrating the RAG pipeline
3. **[IBM Granite Vision Models](https://www.ibm.com/granite/)**: For understanding image content
4. **Visual Grounding**: Docling's unique capability for source attribution


---

## Prerequisites

Before we begin, ensure you have:
- Completed Labs 1 and 2 (or equivalent Docling knowledge)
- Python 3.10, 3.11, or 3.12 installed
- Basic understanding of embeddings and vector databases
- Familiarity with the concepts from previous labs

## Setting up the environment

Ensure you are running Python 3.10, 3.11 or 3.12 in a freshly created virtual environment.

In [1]:
import sys
assert sys.version_info >= (3, 10) and sys.version_info < (3, 13), "Use Python 3.10, 3.11, or 3.12 to run this notebook."

## Install Dependencies

Now let's install all necessary packages. Note that we're building on the packages from previous labs and adding vision and RAG-specific components:

In [2]:
! pip install "git+https://github.com/ibm-granite-community/utils.git" \
    transformers \
    pillow \
    langchain_community \
    langchain_huggingface \
    langchain_milvus \
    docling \
    replicate \
    matplotlib

  Cloning https://github.com/ibm-granite-community/utils.git to /tmp/pip-req-build-3jkhlxr3
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils.git /tmp/pip-req-build-3jkhlxr3
  Resolved https://github.com/ibm-granite-community/utils.git to commit 68309388c1f857d25a6fe680caedb6d4cc1dcdaf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 10.8 MB/s eta 0:00:00
   ━━━━

## Import Required Libraries and Configure Logging


### Logging Configuration

To see detailed information about the document processing and visual grounding operations, we'll configure INFO log level.

NOTE: It is okay to skip running this cell if you prefer less verbose output.

In [3]:
import logging

logging.basicConfig(level=logging.INFO)

### Required Libraries

Now let's import all necessary modules, organized by their purpose:


In [4]:
import json
import base64
import io
import itertools
import tempfile
from pathlib import Path
from tempfile import mkdtemp
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt
import PIL.Image
import PIL.ImageOps
from PIL import ImageDraw
from IPython.display import display

# Docling imports for document processing and visual grounding
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.datamodel.document import DoclingDocument
from docling.chunking import DocMeta
from docling_core.transforms.chunker.hybrid_chunker import HybridChunker
from docling_core.types.doc.document import TableItem, RefItem
from docling_core.types.doc.labels import DocItemLabel

# LangChain imports for RAG pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.llms import Replicate
from langchain_core.documents import Document
from langchain_core.vectorstores import VectorStore
from langchain_milvus import Milvus
from langchain.prompts import PromptTemplate
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Model imports
from transformers import AutoTokenizer, AutoProcessor
from ibm_granite_community.notebook_utils import get_env_var, escape_f_string

---

## Selecting and loading the AI models

### Load the Granite Embeddings Model

We'll use the [Granite Embeddings model](https://huggingface.co/collections/ibm-granite/granite-embedding-models-6750b30c802c1926a35550bb) for generating text embedding vectors.

To use a different embeddings model, you can refer to [this Embeddings Model recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_Embeddings_Models.ipynb).

In [5]:
embeddings_model_path = "ibm-granite/granite-embedding-30m-english"
embeddings_model = HuggingFaceEmbeddings(
    model_name=embeddings_model_path,
)
embeddings_tokenizer = AutoTokenizer.from_pretrained(embeddings_model_path)
print(f"Embeddings model loaded: {embeddings_model_path}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/683 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Embeddings model loaded: ibm-granite/granite-embedding-30m-english


### Load the Granite Vision Model

**Note**: To set up Replicate, see [Getting Started with Replicate](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Getting_Started/Getting_Started_with_Replicate.ipynb).


To connect to a model on a provider other than Replicate, see the [LLM component recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_LLMs.ipynb).

In [6]:
vision_model_path = "ibm-granite/granite-vision-3.2-2b"
vision_model = Replicate(
    model=vision_model_path,
    replicate_api_token=get_env_var("REPLICATE_API_TOKEN"),
    model_kwargs={
        "max_tokens": embeddings_tokenizer.max_len_single_sentence,
        "min_tokens": 100,
        "temperature": 0.01 # low temperature for reproduceability
    },
)
vision_processor = AutoProcessor.from_pretrained(vision_model_path)
print(f"Vision model loaded: {vision_model_path}")

REPLICATE_API_TOKEN loaded from Google Colab secret.


processor_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

Vision model loaded: ibm-granite/granite-vision-3.2-2b


### Load the Granite Language Model


In [7]:
model_path = "ibm-granite/granite-3.3-8b-instruct"
model = Replicate(
    model=model_path,
    replicate_api_token=get_env_var("REPLICATE_API_TOKEN"),
    model_kwargs={
        "max_tokens": 1000,
        "min_tokens": 100,
        "temperature": 0.01 # low temperature for reproduceability

    },
)
tokenizer = AutoTokenizer.from_pretrained(model_path)
print(f"Language model loaded: {model_path}")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

Language model loaded: ibm-granite/granite-3.3-8b-instruct



---


## Enhanced document processing with visual grounding support

### The Foundation of Visual Grounding



In [8]:
# Configure converter with visual grounding capabilities
pdf_pipeline_options = PdfPipelineOptions(
    do_ocr=False,  # Set to True if your PDFs contain scanned images
    generate_picture_images=True,  # Extract images from documents
    generate_page_images=True,  # CRITICAL: Generate page images for visual grounding
)

format_options = {
    InputFormat.PDF: PdfFormatOption(pipeline_options=pdf_pipeline_options),
}

converter = DocumentConverter(format_options=format_options)
print("Document converter configured with visual grounding support")

Document converter configured with visual grounding support


### Create a Document Store for Visual Grounding

In [9]:
# Create document store for visual grounding
doc_store = {}
doc_store_root = Path(mkdtemp())
print(f"Document store created at: {doc_store_root}")

Document store created at: /tmp/tmp41u85bmw


### Convert Documents with Visual Tracking


In [ ]:
sources = [
    "https://midwestfoodbank.org/images/AR_2020_WEB2.pdf",
    # Add more document URLs here as needed
]

conversions = {}

print("Starting document conversion with visual grounding support...")
for source in sources:
    print(f"\n Processing: {source}")

    # Convert document
    result = converter.convert(source=source)
    docling_document = result.document
    conversions[source] = docling_document

    # Save document to store for visual grounding
    # The binary hash ensures unique identification
    file_path = doc_store_root / f"{docling_document.origin.binary_hash}.json"
    docling_document.save_as_json(file_path)
    doc_store[docling_document.origin.binary_hash] = file_path

    print(f"Document converted and saved")
    print(f"  - Document ID: {docling_document.origin.binary_hash}")
    print(f"  - Pages: {len(docling_document.pages)}")
    print(f"  - Tables: {len(docling_document.tables)}")
    print(f"  - Images: {len(docling_document.pictures)}")

Starting document conversion with visual grounding support...

 Processing: https://midwestfoodbank.org/images/AR_2020_WEB2.pdf


## Process Document Content with Visual Grounding Metadata


### Process Text Chunks with Location Tracking


In [ ]:
from docling.chunking import DocMeta

doc_id = 0
texts: list[Document] = []

print("\nProcessing text chunks with visual grounding metadata...")
for source, docling_document in conversions.items():
    chunker = HybridChunker(tokenizer=embeddings_tokenizer)

    for chunk in chunker.chunk(docling_document):
        items = chunk.meta.doc_items

        # Skip single-item chunks that are tables (we'll process them separately)
        if len(items) == 1 and isinstance(items[0], TableItem):
            continue

        refs = " ".join(map(lambda item: item.get_ref().cref, items))
        text = chunk.text

        # Create document with enhanced metadata for visual grounding
        document = Document(
            page_content=text,
            metadata={
                "doc_id": (doc_id:=doc_id+1),
                "source": source,
                "ref": refs,  # References for tracking specific document items
                "dl_meta": chunk.meta.model_dump(),  # CRITICAL: Store chunk metadata for visual grounding
                "origin_hash": docling_document.origin.binary_hash  # Link to stored document
            },
        )
        texts.append(document)

print(f"Created {len(texts)} text chunks with visual grounding metadata")

### Process Tables with Spatial Information

Tables require special handling to preserve their structure and location:

In [ ]:
doc_id = len(texts)
tables: list[Document] = []

print("\nProcessing tables...")
for source, docling_document in conversions.items():
    for table in docling_document.tables:
        if table.label in [DocItemLabel.TABLE]:
            ref = table.get_ref().cref
            text = table.export_to_markdown(docling_document)

            # Extract provenance information for visual grounding
            prov_data = []
            if hasattr(table, 'prov') and table.prov:
                for prov in table.prov:
                    # Get the page to access its height for coordinate conversion
                    if prov.page_no < len(docling_document.pages):
                        page = docling_document.pages[prov.page_no]
                        # Convert to top-left origin and normalize
                        bbox = prov.bbox.to_top_left_origin(page_height=page.size.height)
                        bbox_norm = bbox.normalized(page.size)

                        prov_data.append({
                           "page_no": prov.page_no,
                           "bbox": {
                              "l": bbox_norm.l,  # Use normalized coordinates
                              "t": bbox_norm.t,
                              "r": bbox_norm.r,
                              "b": bbox_norm.b
                        }
                    })

            document = Document(
                page_content=text,
                metadata={
                    "doc_id": (doc_id:=doc_id+1),
                    "source": source,
                    "ref": ref,
                    "origin_hash": docling_document.origin.binary_hash,
                    "item_type": "table",  # Mark as table
                    "prov_data": prov_data  # Store provenance as simple data
                },
            )
            tables.append(document)

print(f"Created {len(tables)} table documents")

### Process Images with Vision Understanding

NOTE: Processing images can take time depending on the number of images and the vision model service. Each image will be analyzed individually.

In [ ]:
def encode_image(image: PIL.Image.Image, format: str = "png") -> str:
    """Encode image to base64 for vision model processing"""
    image = PIL.ImageOps.exif_transpose(image) or image
    image = image.convert("RGB")

    buffer = io.BytesIO()
    image.save(buffer, format)
    encoding = base64.b64encode(buffer.getvalue()).decode("utf-8")
    uri = f"data:image/{format};base64,{encoding}"
    return uri

# Configure vision prompt - feel free to experiment with this!
image_prompt = "Give a detailed description of what is depicted in the image"
conversation = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": image_prompt},
        ],
    },
]
vision_prompt = vision_processor.apply_chat_template(
    conversation=conversation,
    add_generation_prompt=True,
)

pictures: list[Document] = []
doc_id = len(texts) + len(tables)

for source, docling_document in conversions.items():
    for picture in docling_document.pictures:
        ref = picture.get_ref().cref
        print(f"  Processing image: {ref}")

        image = picture.get_image(docling_document)
        if image:
            # Generate image description using vision model
            text = vision_model.invoke(vision_prompt, image=encode_image(image))

            # Extract provenance information for visual grounding
            prov_data = []
            if hasattr(picture, 'prov') and picture.prov:
                for prov in picture.prov:
                    # Get the page to access its height for coordinate conversion
                    if prov.page_no < len(docling_document.pages):
                        page = docling_document.pages[prov.page_no]
                        # Convert to top-left origin and normalize
                        bbox = prov.bbox.to_top_left_origin(page_height=page.size.height)
                        bbox_norm = bbox.normalized(page.size)

                        prov_data.append({
                            "page_no": prov.page_no,
                            "bbox": {
                                "l": bbox_norm.l,
                                "t": bbox_norm.t,
                                "r": bbox_norm.r,
                                "b": bbox_norm.b
                            }
                        })

            document = Document(
                page_content=text,
                metadata={
                    "doc_id": (doc_id:=doc_id+1),
                    "source": source,
                    "ref": ref,
                    "origin_hash": docling_document.origin.binary_hash,
                    "item_type": "picture",  # Mark as picture for special handling
                    "prov_data": prov_data  # Store normalized provenance data
                },
            )
            pictures.append(document)

print(f"Created {len(pictures)} image descriptions")

### Display Sample Processed Documents

Let's examine what we've created to understand the multimodal nature of our system:

In [ ]:
print("\nSample processed documents:")
print("=" * 80)

# Show sample text chunks
print("\nTEXT CHUNK EXAMPLES:")
print("-" * 80)
for i, text_doc in enumerate(texts[:3]):  # Show first 3 text chunks
    print(f"\nText Chunk {i+1}:")
    print(f"  Document ID: {text_doc.metadata['doc_id']}")
    print(f"  Source: {text_doc.metadata['source'].split('/')[-1]}")  # Just filename
    print(f"  Reference: {text_doc.metadata['ref']}")
    print(f"  Has visual grounding: {'dl_meta' in text_doc.metadata}")
    print(f"  Content preview:")
    print(f"    {text_doc.page_content[:250]}...")
    if i < 2:  # Add separator between examples except after the last one
        print("-" * 40)

# Show sample tables
print("\n\nTABLE EXAMPLES:")
print("-" * 80)
if tables:
    for i, table_doc in enumerate(tables[:3]):  # Show first 3 tables
        print(f"\nTable {i+1}:")
        print(f"  Document ID: {table_doc.metadata['doc_id']}")
        print(f"  Reference: {table_doc.metadata['ref']}")
        print(f"  Content preview (Markdown format):")
        # Show first few lines of the table
        table_lines = table_doc.page_content.split('\n')[:8]
        for line in table_lines:
            print(f"    {line}")
else:
    print("  No tables found in the document.")

# Show sample images with descriptions
print("\n\nIMAGE EXAMPLES WITH AI-GENERATED DESCRIPTIONS:")
print("-" * 80)
if pictures:
    for i, pic_doc in enumerate(pictures[:3]):  # Show first 3 images
        print(f"\nImage {i+1}:")
        print(f"  Document ID: {pic_doc.metadata['doc_id']}")
        print(f"  Reference: {pic_doc.metadata['ref']}")
        print(f"  AI-Generated Description:")
        # Wrap the description for better readability
        import textwrap
        wrapped_text = textwrap.fill(pic_doc.page_content, width=70, initial_indent="    ", subsequent_indent="    ")
        print(wrapped_text)

        # Display the actual image
        source = pic_doc.metadata['source']
        ref = pic_doc.metadata['ref']
        docling_document = conversions[source]
        picture = RefItem(cref=ref).resolve(docling_document)
        image = picture.get_image(docling_document)
        if image:
            print(f"\n  Original Image:")
            # Resize image for display if too large
            display_image = image.copy()
            max_width = 600
            if display_image.width > max_width:
                ratio = max_width / display_image.width
                new_height = int(display_image.height * ratio)
                display_image = display_image.resize((max_width, new_height), PIL.Image.Resampling.LANCZOS)
            display(display_image)

        if i < min(2, len(pictures)-1):
            print("-" * 40)
else:
    print("  No images found in the document.")

## Visual Grounding Functions

### Understanding Visual Source Attribution

Visual grounding is what sets this system apart. These functions enable us to:
1. **Locate**: Find the exact source of any retrieved information
2. **Highlight**: Draw visual indicators on document pages
3. **Differentiate**: Use different styles for text, tables, and images
4. **Verify**: Allow users to confirm AI responses against source documents


### Implementing the Core Visual Grounding Function

In [ ]:
def visualize_chunk_grounding(chunk, doc_store, highlight_color="blue"):
    """
    Visualize where a text chunk comes from in the original document.

    This function:
    1. Loads the original document from the store
    2. Finds the pages containing the chunk content
    3. Draws bounding boxes around the source regions
    4. Displays the highlighted pages

    Args:
        chunk: LangChain Document with visual grounding metadata
        doc_store: Dictionary mapping document hashes to file paths
        highlight_color: Color for highlighting (blue, green, red, etc.)

    Returns:
        Dictionary of page images with highlights
    """
    # Get the origin hash
    origin_hash = chunk.metadata.get("origin_hash")
    if not origin_hash:
        print("No origin hash found in metadata")
        return None

    # Load the full document from store
    dl_doc = DoclingDocument.load_from_json(doc_store.get(origin_hash))

    print(f"Visualizing source location for chunk {chunk.metadata.get('doc_id', 'Unknown')}")

    # Handle different types of content
    page_images = {}
    item_type = chunk.metadata.get("item_type", "text")

    if item_type in ["picture", "table"] and "prov_data" in chunk.metadata:
        # Handle tables and pictures with simple provenance data
        prov_data = chunk.metadata["prov_data"]

        if not prov_data:
            print(f"No provenance data available for this {item_type}")
            return None

        for prov in prov_data:
            page_no = prov["page_no"]

            # Get page image
            if page_no < len(dl_doc.pages):
                page = dl_doc.pages[page_no]
                if hasattr(page, 'image') and page.image:
                    if page_no not in page_images:
                        img = page.image.pil_image.copy()
                        page_images[page_no] = {
                            'image': img,
                            'page': page,
                            'draw': ImageDraw.Draw(img)
                        }

                    # Draw bounding box
                    draw = page_images[page_no]['draw']
                    bbox = prov["bbox"]

                    # Draw bounding box
                    draw = page_images[page_no]['draw']
                    bbox = prov["bbox"]

                    # The coordinates are already normalized and in top-left origin
                    # Just scale to image dimensions
                    img_width = page_images[page_no]['image'].width
                    img_height = page_images[page_no]['image'].height

                    l = int(bbox["l"] * img_width)
                    r = int(bbox["r"] * img_width)
                    t = int(bbox["t"] * img_height)
                    b = int(bbox["b"] * img_height)

                    # Ensure coordinates are valid (min/max) just in case
                    l, r = min(l, r), max(l, r)
                    t, b = min(t, b), max(t, b)

                    # Clamp to image bounds
                    l = max(0, min(l, img_width - 1))
                    r = max(0, min(r, img_width - 1))
                    t = max(0, min(t, img_height - 1))
                    b = max(0, min(b, img_height - 1))

                    # Draw highlight with different styles for different types
                    if item_type == "picture":
                        draw.rectangle([l, t, r, b], outline=highlight_color, width=4)
                        draw.text((l, t-20), "IMAGE", fill=highlight_color)
                    elif item_type == "table":
                        draw.rectangle([l, t, r, b], outline=highlight_color, width=3)
                        draw.text((l, t-20), "TABLE", fill=highlight_color)

    elif "dl_meta" in chunk.metadata:
        # Handle text chunks with DocMeta
        try:
            meta = DocMeta.model_validate(chunk.metadata["dl_meta"])

            # Process each item in the chunk to find source locations
            for doc_item in meta.doc_items:
                if hasattr(doc_item, 'prov') and doc_item.prov:
                    for prov in doc_item.prov:
                        page_no = prov.page_no

                        # Get or create page image
                        if page_no not in page_images:
                            if page_no < len(dl_doc.pages):
                                page = dl_doc.pages[page_no]
                                if hasattr(page, 'image') and page.image:
                                    img = page.image.pil_image.copy()
                                    page_images[page_no] = {
                                        'image': img,
                                        'page': page,
                                        'draw': ImageDraw.Draw(img)
                                    }

                        # Draw bounding box on the page
                        if page_no in page_images:
                            page_data = page_images[page_no]
                            page = page_data['page']
                            draw = page_data['draw']

                            # Convert coordinates to image space
                            bbox = prov.bbox.to_top_left_origin(page_height=page.size.height)
                            bbox = bbox.normalized(page.size)

                            # Scale to actual image dimensions
                            l = int(bbox.l * page_data['image'].width)
                            r = int(bbox.r * page_data['image'].width)
                            t = int(bbox.t * page_data['image'].height)
                            b = int(bbox.b * page_data['image'].height)

                            # Draw highlight rectangle
                            draw.rectangle([l, t, r, b], outline=highlight_color, width=2)
        except Exception as e:
            print(f"Error processing text chunk metadata: {e}")
            return None
    else:
        print("No visual grounding metadata available for this chunk")
        return None

    # Display highlighted pages
    for page_no, page_data in sorted(page_images.items()):
        plt.figure(figsize=(12, 16))
        plt.imshow(page_data['image'])
        plt.axis('off')

        # Add title indicating content type
        if item_type == "picture":
            title = "Image Location"
        elif item_type == "table":
            title = "Table Location"
        else:
            title = "Text Location"
        plt.title(f'{title} - Page {page_no + 1}', fontsize=16)
        plt.tight_layout()
        plt.show()

    return page_images


## Populate the Vector Database

### Choose and Configure Vector Database

We'll use Milvus, a high-performance vector database. For other vector database options, see [this Vector Store recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_Vector_Stores.ipynb).

In [ ]:
# Create a temporary database file
db_file = tempfile.NamedTemporaryFile(prefix="vectorstore_", suffix=".db", delete=False).name
print(f"Vector database will be saved to: {db_file}")

# Initialize Milvus vector store
vector_db: VectorStore = Milvus(
    embedding_function=embeddings_model,
    connection_args={"uri": db_file},
    auto_id=True,
    enable_dynamic_field=True,  # Allows flexible metadata storage
    index_params={"index_type": "AUTOINDEX"},  # Automatic index optimization
)


### Add Documents to Vector Database

In [ ]:
print("\nAdding documents to vector database...")
documents = list(itertools.chain(texts, tables, pictures))
ids = vector_db.add_documents(documents)
print(f"Successfully added {len(ids)} documents to the vector database")
print(f"  - Text chunks: {len(texts)}")
print(f"  - Tables: {len(tables)}")
print(f"  - Image descriptions: {len(pictures)}")

## Test Retrieval with Visual Grounding

### Understanding Retrieval Testing

### Basic Retrieval Test


In [ ]:
# Test query
test_query = "How much was spent on food distribution relative to the amount of food distributed?"

print(f"\nTesting retrieval for query: '{test_query}'")
print("=" * 80)

# Retrieve relevant documents
retrieved_docs = vector_db.as_retriever().invoke(test_query)

# Display retrieved documents
for i, doc in enumerate(retrieved_docs):
    print(f"\nRetrieved Document {i+1}:")

    # Determine content type
    item_type = doc.metadata.get('item_type', 'text')
    if item_type == 'picture':
        content_type = "AI-Generated Image Description"
    elif item_type == 'table':
        content_type = "Table (Markdown)"
    else:
        content_type = "Text Chunk"

    print(f"Type: {content_type}")
    print(f"Content preview: {doc.page_content[:200]}...")
    print(f"Source: {doc.metadata['source'].split('/')[-1]}")

## Enhanced RAG with Visual Grounding - Bringing It All Together


### The Complete Multimodal RAG Pipeline

Now we'll implement the full RAG system that:
1. Retrieves relevant multimodal content
2. Shows exactly where each piece comes from
3. Generates accurate, grounded responses

### Understanding Visual Grounding for Different Content Types

Our system handles three content types distinctly:

1. **Text Chunks**: Standard highlighting shows text passages
2. **Tables**: Thick borders with "TABLE" labels mark structured data
3. **Images**: Distinctive borders with "IMAGE" labels show picture locations

This visual differentiation helps users quickly understand what type of content contributed to the AI's response.

### Implementing the Complete RAG Pipeline

In [ ]:
def rag_with_visual_grounding(question, vector_db, doc_store, model, tokenizer, top_k=5):
    """
    Perform RAG with visual grounding of results.

    This function:
    1. Retrieves relevant chunks from the vector database
    2. Visualizes where each chunk comes from in the original document
    3. Generates a response using the retrieved context

    Args:
        question: User's query
        vector_db: Vector database with embedded documents
        doc_store: Document store for visual grounding
        model: Language model for response generation
        tokenizer: Tokenizer for the language model
        top_k: Number of chunks to retrieve

    Returns:
        Tuple of (outputs, relevant_chunks)
    """
    print(f"\nQuestion: {question}")
    print("=" * 80)

    # Step 1: Retrieve relevant chunks
    print(f"\nRetrieving top {top_k} relevant chunks...")
    retriever = vector_db.as_retriever(search_kwargs={"k": top_k})
    relevant_chunks = retriever.invoke(question)

    print(f"Found {len(relevant_chunks)} relevant chunks")

    # Step 2: Visualize each chunk's source location
    print("\nVisualizing source locations:")

    for i, chunk in enumerate(relevant_chunks):
        print(f"\n--- Result {i+1} ---")

        # Determine content type
        item_type = chunk.metadata.get('item_type', 'text')
        if item_type == 'picture':
            content_type = "AI-Generated Image Description"
            color = 'red'
        elif item_type == 'table':
            content_type = "Table (Markdown)"
            color = 'green'
        else:
            content_type = "Text Chunk"
            color = 'blue'

        print(f"Content type: {content_type}")
        print(f"Text preview: {chunk.page_content[:200]}...")
        print(f"Source: {chunk.metadata.get('source', 'Unknown').split('/')[-1]}")

        # Show visual grounding if available
        if "dl_meta" in chunk.metadata or "prov_data" in chunk.metadata:
            visualize_chunk_grounding(
                chunk,
                doc_store,
                highlight_color=color
            )
        else:
            print("  (No visual grounding available for this chunk)")

    # Step 3: Create RAG pipeline for response generation
    print("\nGenerating response with LLM...")

    # Create Granite prompt template
    prompt = tokenizer.apply_chat_template(
        conversation=[{
            "role": "user",
            "content": "{input}",
        }],
        documents=[{
            "doc_id": "0",
            "text": "{context}",
        }],
        add_generation_prompt=True,
        tokenize=False,
    )

    prompt_template = PromptTemplate.from_template(
        template=escape_f_string(prompt, "input", "context")
    )

    # Document prompt template
    document_prompt_template = PromptTemplate.from_template(template="""\
<|end_of_text|>
<|start_of_role|>document {{"document_id": "{doc_id}"}}<|end_of_role|>
{page_content}""")

    # Create chains
    combine_docs_chain = create_stuff_documents_chain(
        llm=model,
        prompt=prompt_template,
        document_prompt=document_prompt_template,
        document_separator="",
    )

    rag_chain = create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=combine_docs_chain,
    )

    # Generate response
    outputs = rag_chain.invoke({"input": question})

    print("\nGenerated Answer:")
    print("=" * 80)
    print(outputs['answer'])
    print("=" * 80)

    return outputs, relevant_chunks

## Demonstrate the Complete System

### Main Query with Visual Grounding


In [ ]:
main_query = "How much was spent compared to the amount of food distributed?"
outputs, chunks = rag_with_visual_grounding(
    main_query,
    vector_db,
    doc_store,
    model,
    tokenizer
)

# Summary and Next Steps

### What You've Accomplished

Congratulations! You've successfully built an advanced multimodal RAG system with visual grounding. Here's what you've learned:

- **Lab 1**: Document structure preservation enables everything else
- **Lab 2**: Intelligent chunking optimizes retrieval quality
- **Lab 3**: Visual grounding transforms RAG into transparent AI


---

## Next Steps: Where to Go from Here

### Immediate Actions

1. **Experiment with Your Documents**
   - Try documents with complex layouts
   - Test with technical diagrams and charts
   - Process multi-page reports with mixed content

2. **Customize for Your Domain**
   ```python
   # Example: Domain-specific image prompts
   medical_prompt = "Describe this medical image, noting any abnormalities or key features"
   financial_prompt = "Analyze this financial chart, identifying trends and key data points"
   ```

3. **Optimize Performance**
   - Batch process documents
   - Implement caching for visual grounding
   - Use GPU acceleration for vision models

---

## Resources for Continued Learning

### Official Documentation
- **[Docling Documentation](https://github.com/docling-project/docling)**: Latest features and updates
- **[IBM Granite Models](https://www.ibm.com/granite/)**: Model cards and capabilities
- **[LangChain Docs](https://python.langchain.com/)**: RAG patterns and best practices
- **[Milvus Documentation](https://milvus.io/docs)**: Vector database optimization

### Community Resources
- Join the Docling community on GitHub
- Share your implementations
- Contribute improvements back to the project

### Related Topics to Explore
- Document Layout Analysis
- Multimodal Embeddings
- Visual Question Answering
- Explainable AI Systems

---

## Final Thoughts

You've completed an incredible journey from basic document conversion to building a sophisticated, transparent AI system. The combination of Docling's document understanding, Granite's AI capabilities, and visual grounding creates a powerful application.

Your multimodal RAG system with visual grounding represents the cutting edge of document AI. Whether you're building for healthcare, legal, financial, or any other domain, you now have the tools to create AI systems that are not just powerful, but trustworthy and transparent.